In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from mss import mss

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import pyautogui
import time
env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Input Extraction(Screen Extraction)
from PIL import ImageGrab
import cv2

In [3]:
cv2.destroyAllWindows()

## Templates

In [4]:

white_template = cv2.imread('templates/white_temp.png',0) 
blue_template = cv2.imread('templates/blue_temp.png',0)
orange_template = cv2.imread('templates/orange_temp.png',0)
green_template = cv2.imread('templates/green_temp.png',0)
#cv2.imshow('image',green_template)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

## Rewards skeleton

## Initial block count

In [23]:
#one scrren
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
sct_img = np.array(sct.grab(bounding_box))
gray = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)

#gray_template = cv2.cvtColor(green_template, cv2.COLOR_BGR2GRAY)
#np_scrn = np.array(sct_img)

#print( gray_template.shape[::-1])
w,h = blue_template.shape[::-1]  #here
result = cv2.matchTemplate(gray,blue_template, cv2.TM_CCOEFF_NORMED) #here

f = set()
loc = np.where(result >=0.9)
for pt in zip(*loc[::-1]):
    cv2.rectangle(sct_img,pt,(pt[0] + w, pt[1] + h),(0,255,0),3)
    sensitivity = 100
    f.add((round(pt[0]/sensitivity), round(pt[1]/sensitivity)))
    
found_count = len(f)    
print("amount of blocks", found_count)
cv2.imshow("result", sct_img)
#cv2.imshow('screen', np.array(sct_img))
cv2.waitKey(0)
cv2.destroyAllWindows()
initial_block_value = found_count
print(initial_block_value)

amount of blocks 27
27


In [6]:
#screen method 1
#img = ImageGrab.grab(bbox=(200,630,850,600)) #bbox specifies specific region (bbox= x,y,width,height *starts top-left)
#img_np = np.array(img) #this is the array obtained from conversion
#frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
#cv2.imshow("test", frame)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


#one scrren
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
#while True:
#7 outputs(mouse positions)
sct_img = sct.grab(bounding_box)
cv2.imshow('screen', np.array(sct_img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
#screen method 2(Current)
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
while True:
    #7 outputs(mouse positions)
    sct_img = sct.grab(bounding_box)
    cv2.imshow('screen', np.array(sct_img))
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break

# Actions Mapping

In [55]:
#10 outputs
def take_actions(action):
    if action == 0:
        pyautogui.moveTo(640, 200, duration = 0.03) 
    elif action == 1:
        pyautogui.moveTo(740, 200, duration = 0.03) 
    elif action == 2:
        pyautogui.moveTo(840, 200, duration = 0.03)
    elif action ==3:
        pyautogui.moveTo(940, 200, duration = 0.03) 
    elif action ==4:
        pyautogui.moveTo(1040, 200, duration = 0.03)
    elif action ==5:
        pyautogui.moveTo(1140, 200, duration = 0.03)
    elif action ==6:
        pyautogui.moveTo(1240, 200, duration = 0.03)
    elif action == 7:
        pyautogui.moveTo(1340, 200, duration = 0.03) 
    elif action == 8:
        pyautogui.moveTo(1440, 200, duration = 0.03)
    elif action == 9:
        pyautogui.moveTo(1500, 200, duration = 0.03)
    else:
        print("Not allowed")

# Experience

In [38]:
from collections import namedtuple
Xp = namedtuple('Experience',
                        ('state', 'action', 'next_state', 'reward'))
Xp_points = Xp(5,6,7,8)
Xp_points

Experience(state=5, action=6, next_state=7, reward=8)

# Network Skeleton

In [39]:
#--------------------------------------DQN(Fully connected)-----------------------------------------#
class DQN (nn.Module):
    def __init__(self,img_height, img_width):
        super().__init__()
        self.input_layer = nn.Linear(in_features = img_height*img_width*3,out_features=124)
        self.layer2      = nn.Linear(in_features = 124, out_features = 132)
        self.layer3      = nn.Linear(in_features = 132, out_features = 132)
        self.layer4      = nn.Linear(in_features = 132, out_features = 100)
        self.output      = nn.Linear(in_features = 100, out_features = 10)
#--------------------forward pass---------------------------------------------------------------------#
    def forward(self,t):
        t = t.flatten(start_dim = 1) #select first layer and flatten
        t = F.relu(self.input_layer(t))
        t = F.relu(self.layer2(t))
        t = F.relu(self.layer3(t))
        t = F.relu(self.layer4(t))
        t = self.output(t)
        #print(t.item())
        return t

## Learning rate and network setup

In [59]:
#lr = 0.01
#policy_net = DQN(600,850).to(device = "cpu")
#target_net = DQN(600,850).to(device = "cpu")
#target_net.load_state_dict(policy_net.state_dict()) #set weights to be the same
#target_net.eval() #not in training mode
#optimizer = optim.Adam(params = policy_net.parameters(), lr=lr)

# Q values setup

In [41]:
class Q_Values():
    device = "cpu"
    
    @staticmethod
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1, index=actions.unsqueeze(-1))
    
    @staticmethod        
    def get_next(target_net, next_states):
        #print("nx",next_states)
        #print("shape", next_states.shape[0])
        final_state_locations = next_states.flatten(start_dim=1).max(dim=1)[0].eq(0).type(torch.bool)
        #print("ft",final_state_locations)
        non_final_state_locations = (final_state_locations == False)
        #print("nf",non_final_state_locations)
        non_final_states = next_states[non_final_state_locations]
        batch_size = next_states.shape[0]
        values = torch.zeros(batch_size).to(Q_Values.device)
        #values = torch.zeros(2).to(Q_Values.device)
        values[non_final_state_locations] = target_net(non_final_states).max(dim=1)[0].detach()
        return values

# Tensor Extraction

In [42]:
def extract_tensors(experiences):
    batch = Xp(*zip(*experiences))
    t1 = torch.cat(batch.state)
    #print("t1",t1)
    t2 = torch.cat(batch.action)
    #print("t2",t2)
    t3 = torch.cat(batch.reward)
    #print("t3",t3)
    t4 = torch.cat(batch.next_state)
    
    
    
    return t1,t2,t3,t4

# Replay Memory

In [43]:
#-----------------------------DEJAVU----------------------------------------------------------#
class ReplayMemory():
    def __init__(self,capacity):   
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            #overwrite old memory
            self.memory[self.push_count%self.capacity] = experience
        self.push_count+=1
    #number of returned memories will be equal to batch size
    def sample(self, batch_size):
        return rand.sample(self.memory,batch_size)
    #sample must be equal to batch size
    def can_provide_sample(self, batch_size):
        return len(self.memory)>=batch_size
    

# Epsilon Decay Rate

In [44]:
class Greed():
    def __init__(self,start,end,decay):
        self.start = start
        self.end = end
        self.decay = decay
    def get_exploration_rate(self,current_step):
        return self.end + (self.start - self.end)*math.exp(-1. * current_step*self.decay)

In [45]:
def transform_screen_data(screen):
        screen = np.ascontiguousarray(screen, dtype=np.float32)/255
        screen = torch.from_numpy(screen).float()
        
        resize = T.Compose([T.ToPILImage(),
                    T.Resize((600, 850)),
                    T.ToTensor()])
        return resize(screen).unsqueeze(0).to(device = "cpu")

In [56]:
memory = ReplayMemory(100000)
gamma = 0.7 #affects network
import random as rand

In [57]:
w,h = blue_template.shape[::-1]  #set template shape


In [60]:
lr = 0.01
policy_net = DQN(600,850).to(device = "cpu")
target_net = DQN(600,850).to(device = "cpu")
target_net.load_state_dict(policy_net.state_dict()) #set weights to be the same
target_net.eval() #not in training mode
optimizer = optim.Adam(params = policy_net.parameters(), lr=lr)

# Main

In [62]:
#def launch():
exp_rate = 1
episode = 0
max_rate = 1
min_rate = 0.01
exp_decay = 0.01
for n in range (200):
    w,h = blue_template.shape[::-1]  #set template shape
    count = 0
    current_blocks = initial_block_value
    past_blocks = initial_block_value
    reward = 0
    bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} 
    sct = mss()
    sct_img =(sct.grab(bounding_box))
    first_example = transform_screen_data(sct_img) #Transform screen(current screen)/previous
    sct_img =(sct.grab(bounding_box))
    second_example = transform_screen_data(sct_img) #Transform screen(current screen)/previous
    sct_img =(sct.grab(bounding_box))
    third_example = transform_screen_data(sct_img) #Transform screen(current screen)/previous
    example = first_example - second_example - third_example
    while True:
        #10 outputs(mouse positions)
        init_random = np.random.uniform(0,1)
        past_blocks = current_blocks
        count+=1
        x,y = pyautogui.position()
        pyautogui.click(x, y)
###############################################STATE########################################################        
        state = example
###############################################ACTION########################################################
        #epsilon greed
        exp_thresh = rand.uniform(0,1)
        if exp_thresh > exp_rate:
             with torch.no_grad():#need to review this
                action = policy_net(example).argmax(dim=1).to(device ="cpu")
                print("net_value", action)
        else:
            rndm = rand.randrange(1,10)
            action = torch.tensor([rndm]).to(device = "cpu")
            print("rand_value", action)
        take_actions(action)
        
#############################################REWARD###########################################################        
        sct_img =(sct.grab(bounding_box))
        gray = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY) #block detection
        f = set()
        result = cv2.matchTemplate(gray,blue_template, cv2.TM_CCOEFF_NORMED) #match template per frame
        loc = np.where(result >=0.9)
        for pt in zip(*loc[::-1]):
            sensitivity = 100
            f.add((round(pt[0]/sensitivity), round(pt[1]/sensitivity)))
        found_count = len(f) # reward factor
        current_blocks = found_count
        #reward feed to network(onl give reard if resulting action is not a black screen)
        if np.average(gray) > 20:
            if(current_blocks < past_blocks):
                print("current_blocks", current_blocks,"past_blocks",past_blocks )
                reward =(past_blocks - current_blocks)
                print("sub",reward)
                reward_tensor = torch.FloatTensor([reward], device = "cpu")
                print("reward_tensor",reward_tensor)
            else:
                reward = 0
                reward_tensor = torch.FloatTensor([reward], device = "cpu")
                print("reward_tensor",reward_tensor)
            
        #from that action i got this state
        sct_img =(sct.grab(bounding_box))
        gray1 = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY) #block detection
#########################################NEXT_STATE############################################################        
        # if resulting action is a black screen
        if(np.average(gray) < 20):
            #reward = 0
            #current_blocks = 0
            #past_blocks = 0
            #reward_tensor = torch.FloatTensor([reward], device = "cpu")
            screen_return = torch.zeros_like(state)
        else:
            second_example1 = transform_screen_data(sct_img)
            sct_img =(sct.grab(bounding_box))
            second_example2 = transform_screen_data(sct_img)
            sct_img =(sct.grab(bounding_box))
            second_example3 = transform_screen_data(sct_img)
            screen_return = second_example1 - second_example2 - second_example3
        next_state = screen_return
        memory.push(Xp(state,action,next_state,reward_tensor))
        state = next_state
######################################EXPERIENCE AND Q VALUES##################################################
        if memory.can_provide_sample(200):
            Xps = memory.sample(200)
            states, actions,rewards,next_states = extract_tensors(Xps)
            current_q_values = Q_Values.get_current(policy_net,states,actions)
            next_q_values = Q_Values.get_next(target_net,next_states)
            target_q_values = (next_q_values *(gamma)) + rewards
            loss = F.mse_loss(current_q_values,target_q_values.unsqueeze(1))
            optimizer.zero_grad() #zero out gradients to avoid turning towarda a certain direction
            loss.backward()
            optimizer.step() #back propagate
        if np.average(gray) < 20:
            #time.sleep(5) #delay
            break
        #cv2.imshow('screen', np.array(sct_img))
        #if (cv2.waitKey(1) & 0xFF) == ord('q'):
            #cv2.destroyAllWindows()
            #break
    if n % 100 == 0:
        print("updated policy")
        target_net.load_state_dict(policy_net.state_dict())
    exp_rate = min_rate + (max_rate - min_rate)*np.exp(-exp_decay * n)

rand_value tensor([5])
reward_tensor tensor([0.])
rand_value tensor([1])
reward_tensor tensor([0.])
rand_value tensor([7])
current_blocks 26 past_blocks 27
sub 1
reward_tensor tensor([1.])
rand_value tensor([9])
reward_tensor tensor([0.])
rand_value tensor([9])
updated policy
rand_value tensor([2])
reward_tensor tensor([0.])
rand_value tensor([3])
current_blocks 26 past_blocks 27
sub 1
reward_tensor tensor([1.])
rand_value tensor([3])
reward_tensor tensor([0.])
rand_value tensor([9])
rand_value tensor([4])
reward_tensor tensor([0.])
rand_value tensor([2])
current_blocks 26 past_blocks 27
sub 1
reward_tensor tensor([1.])
rand_value tensor([2])
reward_tensor tensor([0.])
rand_value tensor([6])
rand_value tensor([6])
reward_tensor tensor([0.])
rand_value tensor([9])
current_blocks 26 past_blocks 27
sub 1
reward_tensor tensor([1.])
rand_value tensor([3])
reward_tensor tensor([0.])
net_value tensor([8])
rand_value tensor([4])
reward_tensor tensor([0.])
rand_value tensor([5])
reward_tensor t

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [ ]:
#launch()